In [65]:
# import torch   #v7.1
# import random
# import math
# import cv2
# import numpy as np
# from pathlib import Path
# from PIL import Image
# import matplotlib.pyplot as plt
# from ultralytics import YOLO
# from torchvision import transforms
# from sklearn.metrics import confusion_matrix
# import seaborn as sns

import torch   #V7.2
import numpy as np
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from ultralytics import YOLO
import seaborn as sns
import cv2
import glob
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from collections import defaultdict


In [ ]:
# class CoralReefClassifier:
#     def __init__(self, yaml_path, pretrained_weights='yolov8n.pt'):
#         """
#         Initialize YOLO model with coral-specific augmentations
        
#         Args:
#             yaml_path (str): Path to dataset YAML config
#             pretrained_weights (str): Initial weights path
#         """
#         self.device = self._setup_device()
#         self.model = YOLO(pretrained_weights).to(self.device)
#         self.yaml_path = yaml_path
#         self.class_names = ['Bleached', 'Dead', 'Healthy']
        
#         # Coral-specific augmentations
#         self.color_transforms = transforms.Compose([
#             transforms.ColorJitter(
#                 brightness=(0.8, 1.2),
#                 contrast=(0.7, 1.3),
#                 saturation=(0.6, 1.4),
#                 hue=(-0.1, 0.1)
#             ),
#             transforms.RandomAdjustSharpness(1.5, p=0.3),
#             transforms.RandomGrayscale(p=0.1)
#         ])

#     def _setup_device(self):
#         """Configure device with CUDA error handling"""
#         try:
#             if torch.cuda.is_available():
#                 torch.backends.cudnn.benchmark = True
#                 return 'cuda'
#             return 'cpu'
#         except RuntimeError as e:
#             print(f"CUDA error: {e}\nFalling back to CPU")
#             return 'cpu'

#     def _apply_coral_augmentations(self, img):
#         """Apply underwater-specific augmentations"""
#         img = self.color_transforms(img)
        
#         # Water surface effects
#         if random.random() < 0.3:
#             img = self._add_water_surface_effect(img)
            
#         # Particulate matter
#         if random.random() < 0.2:
#             img = self._add_particulate_effect(img)
            
#         return img

#     def _add_water_surface_effect(self, img):
#         """Add light refraction patterns"""
#         img = np.array(img)
#         rows, cols = img.shape[:2]
        
#         for i in range(rows):
#             offset = int(5 * math.sin(2*math.pi*i/120))
#             img[i,:] = np.roll(img[i,:], offset, axis=0)
            
#         return Image.fromarray(img)

#     def _add_particulate_effect(self, img):
#         """Add floating particulate matter"""
#         img = np.array(img)
#         h, w = img.shape[:2]
        
#         for _ in range(random.randint(10, 30)):
#             x, y = random.randint(0, w-1), random.randint(0, h-1)
#             size = random.randint(1, 3)
#             color = random.choice([(200,200,200), (150,150,150)])
#             cv2.circle(img, (x,y), size, color, -1)
            
#         return Image.fromarray(img)

#     def train(self, epochs=220, batch_size=8, patience=50):
#         """
#         Train model with coral-specific parameters
        
#         Returns:
#             Trained YOLO model
#         """
#         results = self.model.train(
#             data=self.yaml_path,
#             epochs=epochs,
#             batch=batch_size,
#             imgsz=512,
#             device=self.device,
#             patience=patience,
#             augment=True,
#             hsv_h=0.1,
#             hsv_s=0.7,
#             hsv_v=0.4,
#             translate=0.2,
#             scale=0.3,
#             fliplr=0.5,
#             mosaic=1.0,
#             mixup=0.2,
#             pre_transform=self._apply_coral_augmentations,
#             optimizer='AdamW',
#             lr0=0.0005,
#             lrf=0.005,
#             warmup_epochs=8,
#             weight_decay=0.05,
#             label_smoothing=0.1,
#             dropout=0.1
#         )
#         return self.model

#     def evaluate(self, test_path):
#         """
#         Generate detailed coral health report
        
#         Args:
#             test_path (str): Path to test images
            
#         Returns:
#             dict: Comprehensive evaluation metrics
#         """
#         results = self.model.predict(
#             source=test_path,
#             conf=0.25,
#             augment=False,
#             device=self.device
#         )
        
#         # Initialize metrics collection
#         metrics = {
#             'total_images': 0,
#             'coral_count': 0,
#             'health_distribution': {'Bleached':0, 'Dead':0, 'Healthy':0},
#             'coverage': [],
#             'per_image': []
#         }
        
#         for result in results:
#             img_metrics = self._process_image_result(result)
#             metrics['total_images'] += 1
#             metrics['coral_count'] += img_metrics['coral_count']
#             for cls in self.class_names:
#                 metrics['health_distribution'][cls] += img_metrics['health_distribution'][cls]
#             metrics['coverage'].append(img_metrics['coverage'])
#             metrics['per_image'].append(img_metrics)
            
#         self._generate_visual_report(metrics)
#         return metrics

#     def _process_image_result(self, result):
#         """Extract metrics from single image result"""
#         img_metrics = {
#             'filename': Path(result.path).name,
#             'coral_count': 0,
#             'health_distribution': {'Bleached':0, 'Dead':0, 'Healthy':0},
#             'coverage': 0.0,
#             'bboxes': []
#         }
        
#         if result.boxes and result.masks:
#             img_area = result.orig_shape[0] * result.orig_shape[1]
#             coral_area = 0
            
#             for cls, conf, box, mask in zip(result.boxes.cls, result.boxes.conf, 
#                                           result.boxes.xyxy, result.masks):
#                 cls_name = self.class_names[int(cls)]
#                 img_metrics['health_distribution'][cls_name] += 1
#                 img_metrics['coral_count'] += 1
#                 coral_area += mask.area()
#                 img_metrics['bboxes'].append({
#                     'class': cls_name,
#                     'confidence': conf.item(),
#                     'bbox': box.tolist(),
#                     'area': mask.area().item()
#                 })
            
#             img_metrics['coverage'] = coral_area / img_area
            
#         return img_metrics

#     def _generate_visual_report(self, metrics):
#         """Create visualizations for report"""
#         plt.figure(figsize=(15, 5))
        
#         # Health distribution pie chart
#         plt.subplot(1, 3, 1)
#         plt.pie(metrics['health_distribution'].values(), 
#                 labels=metrics['health_distribution'].keys(),
#                 autopct='%1.1f%%')
#         plt.title('Coral Health Distribution')
        
#         # Coverage histogram
#         plt.subplot(1, 3, 2)
#         plt.hist(metrics['coverage'], bins=20)
#         plt.title('Coral Coverage Distribution')
#         plt.xlabel('Coverage Percentage')
#         plt.ylabel('Image Count')
        
#         # Confidence distribution
#         plt.subplot(1, 3, 3)
#         confidences = [b['confidence'] for img in metrics['per_image'] for b in img['bboxes']]
#         plt.hist(confidences, bins=20, color='green', alpha=0.7)
#         plt.title('Detection Confidence Distribution')
#         plt.xlabel('Confidence Score')
        
#         plt.tight_layout()
#         plt.savefig('coral_health_report.png', dpi=300)
#         plt.close()
        
#         # Save detailed CSV report
#         df = pd.DataFrame(metrics['per_image'])
#         df.to_csv('detailed_coral_analysis.csv', index=False)

# v7.1

In [75]:
#v7.2

class CoralReefAnalyzer:
    def __init__(self, pretrained='yolov8n-seg.pt'):
        torch.cuda.empty_cache()
        self.device = self._configure_device()
        
        # Try loading last.pt first, fallback to pretrained
        try:
            self.model = YOLO("runs/segment/train5/weights/last.pt").to(self.device)
            print("Resuming training from last.pt")
        except:
            self.model = YOLO(pretrained).to(self.device)
            print("Starting new training")
        
        self.class_names = ['Bleached', 'Dead', 'Healthy']
        

    def _configure_device(self):
        """Safe device configuration"""
        try:
            if torch.cuda.is_available():
                torch.backends.cudnn.benchmark = True
                return 'cuda'
            return 'cpu'
        except RuntimeError as e:
            print(f"CUDA Error: {e} - Using CPU")
            return 'cpu'

    def train(self, data, epochs=220, batch_size=8, imgsz=512, resume=False):
        """Efficient training with coral-optimized params"""
        results = self.model.train(
            data=data,
            epochs=epochs,
            batch=batch_size,
            imgsz=imgsz,
            device=self.device,
            #augment=True,
            hsv_h=0.05,  # Hue variation
            hsv_s=0.5,   # Saturation
            hsv_v=0.4,   # Value
            fliplr=0.5,
            optimizer='AdamW',
            lr0=1e-4,
            lrf=1e-3,
            amp=True,  # Mixed precision
            overlap_mask=True,
            # mask_ratio=4,
            weight_decay=0.05,
            warmup_epochs=8,
            patience=50,
            resume=resume,
            pretrained="runs/segment/train3/weights/best.pt",
            augment=False,
            mask_ratio=2,
            single_cls=False
        )
        return results

    def analyze_reef(self, test_dir, conf=0.3):
        """Instance-level analysis with proper metrics"""
        results = self.model.predict(
            source=str(Path(test_dir)),
            conf=conf,
            device=self.device,
            verbose=False
        )

        # Initialize metrics
        metrics = {
            'true_labels': [],
            'pred_labels': [],
            'health': defaultdict(int),
            'coverage': [],
            'skipped_images': 0 , # 🚨 Track mismatched pairs
            'confusion': np.zeros((len(self.class_names), len(self.class_names))),  # ADD THIS
            'total': len(results)  # ADD THIS TOO
        }

        for result in results:
            # Get instance predictions
            pred_classes = result.boxes.cls.cpu().numpy().astype(int)
            true_classes = self._get_instance_true_labels(result.path)
        
            # 🚨 Validate instance alignment
            if len(pred_classes) != len(true_classes):
                print(f"Skipping {Path(result.path).name}: {len(pred_classes)} pred vs {len(true_classes)} true")
                metrics['skipped_images'] += 1
                continue

            # Update confusion matrix HERE
            for t, p in zip(true_classes, pred_classes):
                metrics['confusion'][t, p] += 1  # ACTUAL CONFUSION MATRIX UPDATE
        
            # Update metrics
            metrics['true_labels'].extend(true_classes)
            metrics['pred_labels'].extend(pred_classes)
        
            # Count health distribution
            for cls_idx in pred_classes:
                metrics['health'][self.class_names[cls_idx]] += 1

            # 🚨 FIXED INDENTATION: Calculate coverage PER IMAGE
            if result.masks:
                total_area = result.orig_shape[0] * result.orig_shape[1]
                coral_area = sum(m.data.sum().item() for m in result.masks)
                metrics['coverage'].append(coral_area / total_area)

        # 🚨 SAFE METRIC CALCULATION
        try:
            if len(metrics['true_labels']) == 0:
                raise ValueError("No valid instances for metric calculation")
            
            metrics.update({
                'precision': precision_score(metrics['true_labels'], metrics['pred_labels'], 
                               average='weighted', zero_division=0),
                'recall': recall_score(metrics['true_labels'], metrics['pred_labels'],
                                 average='weighted', zero_division=0),
                'f1': f1_score(metrics['true_labels'], metrics['pred_labels'],
                             average='weighted', zero_division=0),
                'accuracy': accuracy_score(metrics['true_labels'], metrics['pred_labels'])
            })
        except Exception as e:
            print(f"Metric calculation failed: {str(e)}")
            metrics.update({'precision': 0, 'recall': 0, 'f1': 0, 'accuracy': 0})

        # Generate reports
        self._visual_report(metrics)
        self._print_summary(metrics)
    
        return metrics

    def _process_result(self, result):
        """Process detection results safely"""
        metrics = {
            'health': {cls:0 for cls in self.class_names},
            'coverage': None,
            'pred': None
        }
        
        try:
            if result.boxes and result.masks:
                total_area = result.orig_shape[0] * result.orig_shape[1]
                coral_area = 0
                counts = [0]*3
                
                for cls, mask in zip(result.boxes.cls.cpu().numpy(), result.masks):
                    cls_idx = int(cls)
                    counts[cls_idx] += 1
                    coral_area += mask.data.sum().item()
                
                metrics['coverage'] = coral_area / total_area if total_area > 0 else 0
                metrics['pred'] = np.argmax(counts)
                    
                for i, cls in enumerate(self.class_names):
                    metrics['health'][cls] = counts[i]
                    
        except Exception as e:
            print(f"Processing error: {e}")
            
        return metrics

    def _get_instance_true_labels(self, img_path):
        """Match labels using base name before .rf. hash"""
        img_path = Path(img_path)
    
        # Extract base name before first .rf.
        base_part = img_path.name.split(".rf.")[0]
        label_dir = img_path.parent.parent / "labels"
    
        # Find first matching label file
        label_files = list(label_dir.glob(f"{base_part}.rf.*.txt"))
    
        if not label_files:
            return []
    
        # Always use first match if multiple exist
        label_path = label_files[0]
    
        true_labels = []
        try:
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) >= 5:  # class_id + bbox + polygon
                        true_labels.append(int(parts[0]))
        except Exception as e:
            print(f"Error reading {label_path.name}: {str(e)}")
    
        return true_labels

    def _visual_report(self, metrics):
        """Generate visual analysis report"""
        plt.figure(figsize=(18,6))
        
        # Confusion Matrix
        plt.subplot(1,2,1)
        sns.heatmap(
            metrics['confusion'], 
            annot=True, 
            fmt='.0f',
            xticklabels=self.class_names,
            yticklabels=self.class_names,
            cmap='Blues',
            annot_kws={"size": 14, "weight": "bold"},  # Larger, clearer numbers
            cbar=False
        )
        plt.title('Coral Health Confusion Matrix\n', fontsize=14, pad=20)
        plt.xlabel('Predicted Class', fontsize=12)
        plt.ylabel('True Class', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
    
        # Health Distribution
        plt.subplot(1, 2, 2)
        plt.bar(
            self.class_names,
            [metrics['health'][cls] for cls in self.class_names],
            color=['lightcoral', 'gray', 'mediumaquamarine'],
            edgecolor='black'
        )
        plt.title('Coral Health Distribution\n', fontsize=14)
        plt.ylabel('Count', fontsize=12)
        plt.xticks(fontsize=12)
        
        plt.tight_layout()
        plt.savefig('coral_health_report.png', dpi=300)
        plt.close()

    def _print_summary(self, metrics):
        """Print enhanced console summary with metrics"""
        total_corals = sum(metrics['health'].values())
    
        print("\n=== Coral Reef Analysis Report ===")
        print(f"Images Analyzed: {metrics['total']}")
        print(f"Total Coral Colonies Detected: {total_corals}")
    
        print("\nClassification Metrics:")
        print(f"- Precision (Weighted): {metrics['precision']:.2%}")
        print(f"- Recall (Weighted):    {metrics['recall']:.2%}")
        print(f"- F1-Score (Weighted):  {metrics['f1']:.2%}")
        print(f"- Accuracy:             {metrics['accuracy']:.2%}")
    
        print("\nHealth Distribution:")
        for cls in self.class_names:
            count = metrics['health'][cls]
            print(f"- {cls}: {count} ({count/total_corals:.1%})" if total_corals > 0 else f"- {cls}: 0")
    
        print(f"\nAverage Coral Coverage: {np.mean(metrics['coverage']):.1%}")
        print(f"Detection Confidence Threshold: {self.model.predictor.args.conf}")
        
    def show_masks(self, image_path):
        results = self.model.predict(image_path)
        for result in results:
            masks = result.masks
            img = result.plot()  # Automatically plots masks
            cv2.imshow("Segmentation", img)
            cv2.waitKey(0)

In [57]:
# Usage
# if __name__ == "__main__":
#     analyzer = CoralReefAnalyzer(
#         pretrained='yolov8n-seg.pt'
#     )

#     print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.2f}GB")

#     # Your dataset root (no need to modify YAML)
#     data_root = r"C:\Users\Rugved\Downloads\marjan-segmentaion.v15i.yolov8"
    
#     # Path to your existing YAML (keep it unchanged)
#     yaml_path = r"C:\Users\Rugved\Downloads\marjan-segmentaion.v15i.yolov8\data.yaml"
    
#     print("Training coral health model...")
#     # analyzer.train(epochs=220)

#     try:
#         analyzer.train(
#             data=yaml_path,
#             epochs=135,
#             batch_size=16,
#             imgsz=512,
#             resume=True
#         )
#     except Exception as e:
#         print(f"Error: {e}\nRetrying with safer settings...")
#         analyzer.train(
#             data=yaml_path,
#             epochs=50,
#             batch_size=4,
#             imgsz=480,
#             resume=True
#         )
    
#     # print("\nAnalyzing reef health...")
#     # metrics = analyzer.analyze_reef(test_dir=Path(data_root)/"test/images")

#     # #analyzer.show_masks("test_image.jpg")  # Visual verification

In [77]:
#Cell 3
# Initialize analyzer with trained weights
analyzer = CoralReefAnalyzer(
    pretrained='runs/segment/train5/weights/best.pt'  # Path to your trained weights
)

Resuming training from last.pt


In [69]:
# Cell 4: Dataset Paths (Modify accordingly)
data_root = Path(r"C:\Users\Rugved\Downloads\marjan-segmentaion.v15i.yolov8")
yaml_path = data_root/"data.yaml"
test_dir = data_root/"test/images"

In [85]:
# Cell 5: Model Testing and Reporting
print("\nAnalyzing reef health...")
metrics = analyzer.analyze_reef(
    test_dir=test_dir,
    conf=0.48  # Adjust confidence threshold as needed
)


Analyzing reef health...
Skipping 02exuae18r-JPG-800x0_q85_jpg.rf.3e57a2f14e62dc6fcfec54e4b76b2353.jpg: 4 pred vs 3 true
Skipping 107_png_jpg.rf.295156cf15525ec323b8938d8e3ca6ba.jpg: 3 pred vs 5 true
Skipping 11023816893_9994f554e2_o_jpg.rf.1b0dce9382768baf7ac09c732b4de67b.jpg: 2 pred vs 3 true
Skipping 11333883796_e74aa9bd17_o_jpg.rf.06960501ef46e5e1d2721c2539a466a0.jpg: 4 pred vs 11 true
Skipping 115_jpg.rf.19e02b603036b792848ddb2d1bed38a5.jpg: 4 pred vs 3 true
Skipping 122_png_jpg.rf.3d69c13cb48c7c73e355ac14cd6d1533.jpg: 0 pred vs 2 true
Skipping 129_png_jpg.rf.177802fe0877de3c99d4fe96cfecc2b3.jpg: 0 pred vs 1 true
Skipping 129_png_jpg.rf.18222340ce6ef46f8f459c441366ef7f.jpg: 0 pred vs 1 true
Skipping 129_png_jpg.rf.385dfba37326d5da0ac857a258bc1d36.jpg: 4 pred vs 1 true
Skipping 129_png_jpg.rf.e9dab777a541ab0b9c80de832e03e3bc.jpg: 2 pred vs 1 true
Skipping 12_png_jpg.rf.6ef7b5a3ead51a94f8c4e2d924d5725a.jpg: 0 pred vs 1 true
Skipping 131_png_jpg.rf.c1a9158627b12633a2064fb514231ddc.j

In [ ]:
# Cell 5: Visual Verification (Optional)
analyzer.show_masks(r"C:\Users\Rugved\Downloads\marjan-segmentaion.v15i.yolov8\test\images\OceanImageBank_FabriceDudenhofer_10_jpg.rf.5268d62410571329985725f4c54b195f.jpg")  # Visual inspection of predictions

Last saved epoch: 0


In [13]:
import torch
print(torch.__version__)  # Should show 2.0.0+cu117 or similar
print(torch.cuda.is_available())  # Should be True if using GPU

2.5.1+cu121
True


In [ ]:
# Run this in a new cell
analyzer.stop_training()  # If using the signal handler method

In [9]:
import torch

# Load last checkpoint
ckpt_path = 'runs/segment/train5/weights/last.pt'
ckpt = torch.load(ckpt_path)

# Preserve progress but change epoch limit
ckpt['train_args']['epochs'] = 135  # New total epochs
ckpt['start_epoch'] = ckpt['epoch']  # Continue from last saved epoch
torch.save(ckpt, ckpt_path)